In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/cs-4780-covid-case-hunters/covid_dataset.pkl
/kaggle/input/cs-4780-covid-case-hunters/load_data.py
/kaggle/input/cs-4780-covid-case-hunters/submit.py
/kaggle/input/cs-4780-covid-case-hunters/dummy_predictions.csv


In [2]:
import pickle
import pandas as pd 
from sklearn import tree
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor, ExtraTreesRegressor, GradientBoostingRegressor
import xgboost as xgb
from sklearn.tree import DecisionTreeRegressor
import lightgbm as lgb
from lightgbm import LGBMRegressor
from sklearn.model_selection import cross_val_score
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.model_selection import GridSearchCV, KFold
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder,MinMaxScaler

In [3]:
F = open(r'/kaggle/input/cs-4780-covid-case-hunters/covid_dataset.pkl','rb')
covid_data = pickle.load(F)
#covid_data
X_train,y_train_log_pos_cases = covid_data['X_train'], covid_data['y_train_log_pos_cases'] 
#X_train: 91 x 23;  y_train_log_pos_cases:91 x 1
X_val, y_val_log_pos_cases = covid_data['X_val'], covid_data['y_val_log_pos_cases'] 
# X_val: 31 x 23; y_val_log_pos_cases: 31 x 1
X_test = covid_data['X_test'] 
# X_test: 61 x 23

In [4]:
X_train = pd.DataFrame(X_train)
X_val = pd.DataFrame(X_val)
X_test = pd.DataFrame(X_test)
X_train.columns = [u'Region',u'Population',u'Area',u'Pop.Density',u'Coastline',u'Net_migration',u'Infant_mortality',
                  u'GDP',u'Literacy',u'Phones',u'Arable',u'Crops',u'Other',u'Birthrate',u'Deathrate',
                  u'Agriculture',u'Industry',u'Service',u'Handwashing_Facilities',u'Extreme_poverty',u'Median_age',u'Life_expectancy',
                   u'Human_development_index']  
X_val.columns = [u'Region',u'Population',u'Area',u'Pop.Density',u'Coastline',u'Net_migration',u'Infant_mortality',
                  u'GDP',u'Literacy',u'Phones',u'Arable',u'Crops',u'Other',u'Birthrate',u'Deathrate',
                  u'Agriculture',u'Industry',u'Service',u'Handwashing_Facilities',u'Extreme_poverty',u'Median_age',u'Life_expectancy',
                   u'Human_development_index'] 
X_test.columns = [u'Region',u'Population',u'Area',u'Pop.Density',u'Coastline',u'Net_migration',u'Infant_mortality',
                  u'GDP',u'Literacy',u'Phones',u'Arable',u'Crops',u'Other',u'Birthrate',u'Deathrate',
                  u'Agriculture',u'Industry',u'Service',u'Handwashing_Facilities',u'Extreme_poverty',u'Median_age',u'Life_expectancy',
                   u'Human_development_index'] 

In [5]:
X_train=pd.DataFrame(X_train,dtype=np.float)
X_val=pd.DataFrame(X_val,dtype=np.float)
X_test=pd.DataFrame(X_test,dtype=np.float)

## Data Cleaing

In [6]:
##median填充缺失值；
# X_train['Literacy'] = X_train['Literacy'].fillna(X_train['Literacy'].median())
# X_train['Phones'] = X_train['Phones'].fillna(X_train['Phones'].median())
# X_train['Birthrate'] = X_train['Birthrate'].fillna(X_train['Birthrate'].median())
# X_train['Deathrate'] = X_train['Deathrate'].fillna(X_train['Deathrate'].median())
# X_train['Industry'] = X_train['Industry'].fillna(X_train['Industry'].median())
# X_train['Service'] = X_train['Service'].fillna(X_train['Service'].median())
# X_train['Handwashing_Facilities'] = X_train['Handwashing_Facilities'].fillna(X_train['Handwashing_Facilities'].median())
# X_train['Extreme_poverty'] = X_train['Extreme_poverty'].fillna(X_train['Extreme_poverty'].median())
# X_train['Median_age'] = X_train['Median_age'].fillna(X_train['Median_age'].median())
# X_train['Human_development_index'] = X_train['Human_development_index'].fillna(X_train['Human_development_index'].median())

# X_val['Agriculture'] = X_val['Agriculture'].fillna(X_val['Agriculture'].median())
# X_val['Industry'] = X_val['Industry'].fillna(X_val['Industry'].median())
# X_val['Service'] = X_val['Service'].fillna(X_val['Service'].median())
# X_val['Handwashing_Facilities'] = X_val['Handwashing_Facilities'].fillna(X_val['Handwashing_Facilities'].median())
# X_val['Extreme_poverty'] = X_val['Extreme_poverty'].fillna(X_val['Extreme_poverty'].median())
# X_val['Median_age'] = X_val['Median_age'].fillna(X_val['Median_age'].median())
# X_val['Human_development_index'] = X_val['Human_development_index'].fillna(X_val['Human_development_index'].median())

# X_test['Literacy'] = X_test['Literacy'].fillna(X_test['Literacy'].median())
# X_test['Handwashing_Facilities'] = X_test['Handwashing_Facilities'].fillna(X_test['Handwashing_Facilities'].median())
# X_test['Extreme_poverty'] = X_test['Extreme_poverty'].fillna(X_test['Extreme_poverty'].median())
# X_test['Median_age'] = X_test['Median_age'].fillna(X_test['Median_age'].median())

In [7]:
X_train['Literacy'] = X_train['Literacy'].fillna(X_train['Literacy'].mean())
X_train['Phones'] = X_train['Phones'].fillna(X_train['Phones'].mean())
X_train['Birthrate'] = X_train['Birthrate'].fillna(X_train['Birthrate'].mean())
X_train['Deathrate'] = X_train['Deathrate'].fillna(X_train['Deathrate'].mean())
X_train['Industry'] = X_train['Industry'].fillna(X_train['Industry'].mean())
X_train['Service'] = X_train['Service'].fillna(X_train['Service'].mean())
X_train['Handwashing_Facilities'] = X_train['Handwashing_Facilities'].fillna(X_train['Handwashing_Facilities'].mean())
X_train['Extreme_poverty'] = X_train['Extreme_poverty'].fillna(X_train['Extreme_poverty'].mean())
X_train['Median_age'] = X_train['Median_age'].fillna(X_train['Median_age'].mean())
X_train['Human_development_index'] = X_train['Human_development_index'].fillna(X_train['Human_development_index'].mean())

X_val['Agriculture'] = X_val['Agriculture'].fillna(X_val['Agriculture'].mean())
X_val['Industry'] = X_val['Industry'].fillna(X_val['Industry'].mean())
X_val['Service'] = X_val['Service'].fillna(X_val['Service'].mean())
X_val['Handwashing_Facilities'] = X_val['Handwashing_Facilities'].fillna(X_val['Handwashing_Facilities'].mean())
X_val['Extreme_poverty'] = X_val['Extreme_poverty'].fillna(X_val['Extreme_poverty'].mean())
X_val['Median_age'] = X_val['Median_age'].fillna(X_val['Median_age'].mean())
X_val['Human_development_index'] = X_val['Human_development_index'].fillna(X_val['Human_development_index'].mean())

X_test['Literacy'] = X_test['Literacy'].fillna(X_test['Literacy'].mean())
X_test['Handwashing_Facilities'] = X_test['Handwashing_Facilities'].fillna(X_test['Handwashing_Facilities'].mean())
X_test['Extreme_poverty'] = X_test['Extreme_poverty'].fillna(X_test['Extreme_poverty'].mean())
X_test['Median_age'] = X_test['Median_age'].fillna(X_test['Median_age'].mean())

In [8]:
## correlation, shape

In [9]:
encoder = OneHotEncoder(sparse=False)
encoderd_train_feature = encoder.fit_transform(pd.DataFrame(X_train['Region']))
encoderd_val_feature = encoder.transform(pd.DataFrame(X_val['Region']))
encoderd_test_feature = encoder.transform(pd.DataFrame(X_test['Region']))
encoderd_train_feature =pd.DataFrame(encoderd_train_feature)
encoderd_val_feature =pd.DataFrame(encoderd_val_feature)
encoderd_test_feature =pd.DataFrame(encoderd_test_feature)

In [10]:
scaler = MinMaxScaler()
scalerd_train_feature = scaler.fit_transform(X_train.iloc[:,1:23])
scalerd_val_feature = scaler.transform(X_val.iloc[:,1:23])
scalerd_test_feature = scaler.transform(X_test.iloc[:,1:23])
scalerd_train_feature =pd.DataFrame(scalerd_train_feature)
scalerd_val_feature =pd.DataFrame(scalerd_val_feature)
scalerd_test_feature =pd.DataFrame(scalerd_test_feature)

In [11]:
scalerd_train_feature.columns = [u'Population',u'Area',u'Pop.Density',u'Coastline',u'Net_migration',u'Infant_mortality',
                  u'GDP',u'Literacy',u'Phones',u'Arable',u'Crops',u'Other',u'Birthrate',u'Deathrate',
                  u'Agriculture',u'Industry',u'Service',u'Handwashing_Facilities',u'Extreme_poverty',u'Median_age',u'Life_expectancy',
                   u'Human_development_index']  
scalerd_val_feature.columns = [u'Population',u'Area',u'Pop.Density',u'Coastline',u'Net_migration',u'Infant_mortality',
                  u'GDP',u'Literacy',u'Phones',u'Arable',u'Crops',u'Other',u'Birthrate',u'Deathrate',
                  u'Agriculture',u'Industry',u'Service',u'Handwashing_Facilities',u'Extreme_poverty',u'Median_age',u'Life_expectancy',
                   u'Human_development_index'] 
scalerd_test_feature.columns = [u'Population',u'Area',u'Pop.Density',u'Coastline',u'Net_migration',u'Infant_mortality',
                  u'GDP',u'Literacy',u'Phones',u'Arable',u'Crops',u'Other',u'Birthrate',u'Deathrate',
                  u'Agriculture',u'Industry',u'Service',u'Handwashing_Facilities',u'Extreme_poverty',u'Median_age',u'Life_expectancy',
                   u'Human_development_index'] 

In [12]:
X_train = pd.concat([encoderd_train_feature,scalerd_train_feature],axis=1)
X_val = pd.concat([encoderd_val_feature,scalerd_val_feature],axis=1)
X_test = pd.concat([encoderd_test_feature,scalerd_test_feature],axis=1)
#Train = pd.concat([X_train,y_train_log_pos_cases],axis=1)
#Train.rename(columns={0:'y_train_log_pos_cases'},inplace=True)
#Val = pd.concat([X_val,y_val_log_pos_cases],axis=1)
#Val.rename(columns={0:'y_val_log_pos_cases'},inplace=True)

In [13]:
#feature selection
#columns_sel = [0, 1, 2, 4, 5, 7, 8, 9, 'Area', 'Pop.Density',  'Population', 3, 'Crops', 10, 6,  'Agriculture', 'Infant_mortality', 'Deathrate', 'Extreme_poverty', 'Coastline', 'Handwashing_Facilities','Net_migration', 'Arable','Median_age','Life_expectancy', 'Human_development_index','Birthrate']
#columns_sel = []
# X_train_sel = X_train[columns_sel]
# X_val_sel = X_val[columns_sel]
# X_test_sel = X_test[columns_sel]
# #X_train_sel.columns = [u'0', u'1', u'2', u'4', u'5', u'7', u'8', u'9',u'Area', u'Pop.Density',  u'Population', u'3', u'Crops', u'10', u'6',  u'Agriculture', u'Infant_mortality', u'Deathrate', u'Extreme_poverty', u'Coastline', u'Handwashing_Facilities',
#                        u'Net_migration', u'Arable', u'Median_age', u'Life_expectancy', u'Human_development_index', u'Birthrate']

In [14]:
# X_train_sel.columns = [u'Area', u'Pop.Density',  u'Population', u'3', u'Crops', u'10', u'6',  u'Agriculture', u'Infant_mortality', u'Deathrate', u'Extreme_poverty', u'Coastline', u'Handwashing_Facilities',
#                        u'Net_migration', u'Arable', u'Median_age', u'Life_expectancy', u'Human_development_index', u'Birthrate']


In [15]:
##xgb 没选feature best so far
# xg_reg = xgb.XGBRegressor(n_estimators=200, learning_rate=0.07, gamma=0, subsample=0.75,colsample_bytree=1, max_depth=7,
#                           eval_metric = 'rmse', objective = 'reg:squarederror')
# xg_reg.fit(X_train, y_train_log_pos_cases,eval_metric='rmse', verbose=True)
# print(xg_reg.score(X_val, y_val_log_pos_cases))
# test_pred = xg_reg.predict(X_test)

In [16]:
xg_reg3 = xgb.XGBRegressor(n_estimators=900, learning_rate=0.01,  max_depth=10, 
                          eval_metric = 'rmse', objective = 'reg:squarederror', subsample = 0.25) #, gamma=0, colsample_bytree=1
xg_reg3.fit(X_train, y_train_log_pos_cases,eval_metric='rmse', verbose=True)
print(xg_reg3.score(X_val, y_val_log_pos_cases))
test_pred3 = xg_reg3.predict(X_test)

0.8526921678476305


In [17]:
##best
# rt_reg = RandomForestRegressor(random_state=233,max_features=16, n_estimators=19)
# rt_reg.fit(X_train_sel, y_train_log_pos_cases)
# print(rt_reg.score(X_val_sel, y_val_log_pos_cases))
# test_pred = rt_reg.predict(X_test_sel)

In [18]:
# rt_reg = RandomForestRegressor(random_state=42)
# rt_reg.fit(X_train_sel, y_train_log_pos_cases)
# print(rt_reg.score(X_val_sel, y_val_log_pos_cases))
# test_pred = rt_reg.predict(X_test_sel)

## Modeling

In [19]:
# X_train

In [20]:
# lgbm_reg = LGBMRegressor(n_estimators = 300, class_weight = 'balanced', num_leaves = 2, max_depth = 2, boosting_type='gbdt',learning_rate=0.05)
# lgbm_reg.fit(X_train, y_train_log_pos_cases)
# print(lgbm_reg.get_params())
# scores = cross_val_score(lgbm_reg, X_val,y_val_log_pos_cases, cv=5, scoring='r2')
# print("%0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))
# print(lgbm_reg.score(X_val,y_val_log_pos_cases))
# test_pred = lgbm_reg.predict(X_test)

In [21]:
#test_pred

In [22]:
import pandas as pd 
#save the prediction array test_pred into a valid submission file
pd.DataFrame(test_pred3).to_csv("submission.csv", header=["cases"], index_label="id")